## Import Google earth engine API and Authenticate

In [9]:
# import Google earth engine module
import ee

#Authenticate the Google earth engine with google account
# First firt setup only, no need to run this after first run

#'4/1AfJohXlW8i8f10hHcz7lAX9RUDx8WIFcqzXcErwbwQzaV6mqI5QackCrfGU' --> something like this, print the token on the menu above
ee.Authenticate()

# for normal/regular use for authorization
# this is required regularly
ee.Initialize()


Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_doiqkQG3NJ1t8IS?source=API


In [55]:
#Pandas modules to interact data
import numpy as np
import pandas as pd
import geopandas as gpd
import json
from pyowm.utils.geo import Polygon as GeoPolygon

import warnings
warnings.filterwarnings('ignore')

with open('../data/geojson/circoscrizioni.geojson') as f:
    circoscrizioni = json.load(f)

In [56]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [58]:
# Longer the duration, Longer it will take for Processing and downloading of Images
StartDate = '2023-01-01'
EndDate = '2023-11-01'

NDVI_circoscrizioni = {}

for feature in circoscrizioni['features']:
    current_coords = feature['geometry']['coordinates']
    pol = ee.Geometry.Polygon(current_coords)

    ens = [
        ee.Feature(pol, {'name': 'nepal-crop-poly', 'id':1})
    ]

    col =  ee.FeatureCollection(ens)

    def extractMinMaxMean(image):
        return image.reduceRegions(**{
        'collection':col,
        'reducer':ee.Reducer.minMax().combine(ee.Reducer.mean(), '', True), 
    })

    # add satellite time series: MODIS NDVI 250m 16 day -------------
    # terra sensor
    collectionModNDVI_terra = ee.ImageCollection('MODIS/006/MOD13Q1').filterDate(StartDate,EndDate) \
        .filterBounds(pol)\
        .select('NDVI')

    # Aqua sensor
    collectionModNDVI_aqua = ee.ImageCollection('MODIS/061/MYD13Q1').filterDate(StartDate,EndDate) \
        .filterBounds(pol)\
        .select('NDVI');

    collectionModNDVI = collectionModNDVI_terra.merge(collectionModNDVI_aqua)
    # THis will provide us 250 m of NDVI datasets from MODIS on 8 day interval

    ModNDVI = collectionModNDVI.filterBounds(pol).map(addDate).map(extractMinMaxMean).flatten()

    column_df = ['NDVI_mean', 'NDVI_min', 'NDVI_max', 'date_min', 'id', 'name']
    
    nested_list = ModNDVI.reduceColumns(ee.Reducer.toList(len(column_df)), column_df).values().get(0)
    data = nested_list.getInfo()

    # dont forget we need to call the callback method "getInfo" to retrieve the data
    df = pd.DataFrame(data, columns=column_df)

    df['date']= pd.to_datetime(df['date_min'], format='%Y%m%d')
    df_new = df.sort_values(by=['date'])

    # put inside NDVI_circoscrizioni dictionary the name of the circoscrizione and the mean of the NDVI_mean column
    NDVI_circoscrizioni[feature['properties']['nome']] = df_new['NDVI_mean'].mean()


In [60]:
# save the dictionary as a json file
with open('../data/NDVI_circoscrizioni.json', 'w') as fp:
    json.dump(NDVI_circoscrizioni, fp)

# Totorial: https://github.com/krishnakafle/Blog_kaflekrishna/blob/main/GEE/min-max-mean-graph-polygon-ndvi/min-max-mean-visualizer.ipynb

## Define starting and ending year of study (Both year Inclusive)

In [1]:
# Longer the duration, Longer it will take for Processing and downloading of Images
StartDate = '2023-01-01'
EndDate = '2023-11-01'

## Define area of interest

In [ ]:
pol = ee.Geometry.Polygon(
    [[[81.03752257234235,28.560898024907527],
      [81.04466797715803,28.560898024907527],
      [81.04466797715803,28.56510070523194],
      [81.03752257234235,28.56510070523194],
      [81.03752257234235,28.560898024907527]
    ]])

In [ ]:
ens = [
    ee.Feature(pol, {'name': 'nepal-crop-poly', 'id':1})
]

col =  ee.FeatureCollection(ens)
print(col.getInfo())

{'type': 'FeatureCollection', 'columns': {'id': 'Integer', 'name': 'String', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[81.03752257234235, 28.560898024907527], [81.04466797715803, 28.560898024907527], [81.04466797715803, 28.56510070523194], [81.03752257234235, 28.56510070523194], [81.03752257234235, 28.560898024907527]]]}, 'id': '0', 'properties': {'id': 1, 'name': 'nepal-crop-poly'}}]}


In [ ]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [ ]:
# add satellite time series: MODIS NDVI 250m 16 day -------------
# terra sensor
collectionModNDVI_terra = ee.ImageCollection('MODIS/006/MOD13Q1').filterDate(StartDate,EndDate) \
    .filterBounds(pol)\
    .select('NDVI')

# Aqua sensor
collectionModNDVI_aqua = ee.ImageCollection('MODIS/061/MYD13Q1').filterDate(StartDate,EndDate) \
    .filterBounds(pol)\
    .select('NDVI');

collectionModNDVI = collectionModNDVI_terra.merge(collectionModNDVI_aqua)
# THis will provide us 250 m of NDVI datasets from MODIS on 8 day interval

In [ ]:
def extractMinMaxMean(image):
    return image.reduceRegions(**{
    'collection':col,
    'reducer':ee.Reducer.minMax().combine(ee.Reducer.mean(), '', True), 
  })

In [ ]:
ModNDVI = collectionModNDVI.filterBounds(pol).map(addDate).map(extractMinMaxMean).flatten()

In [ ]:
sample_result = ModNDVI.first().getInfo()
sample_result

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[81.03752257234235, 28.560898024907527],
    [81.04466797715803, 28.560898024907527],
    [81.04466797715803, 28.56510070523194],
    [81.03752257234235, 28.56510070523194],
    [81.03752257234235, 28.560898024907527]]]},
 'id': '1_2023_01_01_0',
 'properties': {'NDVI_max': 7516,
  'NDVI_mean': 5989.6186440677975,
  'NDVI_min': 4776,
  'date_max': 20230101,
  'date_mean': 20230101,
  'date_min': 20230101,
  'id': 1,
  'name': 'nepal-crop-poly'}}

In [ ]:
# extract the properties column from feature collection
# column order may not be as our sample data order
column_df = ['NDVI_mean', 'NDVI_min', 'NDVI_max', 'date_min', 'id', 'name']

In [ ]:
nested_list = ModNDVI.reduceColumns(ee.Reducer.toList(len(column_df)), column_df).values().get(0)
data = nested_list.getInfo()
data

[[5989.6186440677975, 4776, 7516, 20230101, 1, 'nepal-crop-poly'],
 [6393.663624511082, 5851, 7210, 20230117, 1, 'nepal-crop-poly'],
 [7758.35332464146, 7499, 7895, 20230202, 1, 'nepal-crop-poly'],
 [5087.623207301173, 4456, 5986, 20230109, 1, 'nepal-crop-poly'],
 [7439.797913950457, 7134, 7748, 20230125, 1, 'nepal-crop-poly'],
 [7278.266623207302, 6862, 7752, 20230210, 1, 'nepal-crop-poly'],
 [6741.7874837027375, 6524, 7110, 20230226, 1, 'nepal-crop-poly'],
 [6186.42372881356, 6118, 6358, 20230314, 1, 'nepal-crop-poly'],
 [3075.433507170796, 2865, 3519, 20230330, 1, 'nepal-crop-poly'],
 [2388.8943937418517, 2259, 2826, 20230415, 1, 'nepal-crop-poly'],
 [2110.1610169491523, 2002, 2400, 20230501, 1, 'nepal-crop-poly'],
 [1944.0013037809647, 1777, 2469, 20230517, 1, 'nepal-crop-poly'],
 [1669.2105606258149, 1555, 2291, 20230602, 1, 'nepal-crop-poly'],
 [2400.857235984355, 2010, 3191, 20230618, 1, 'nepal-crop-poly'],
 [6280.444589308996, 6025, 6650, 20230704, 1, 'nepal-crop-poly'],
 [7593

In [ ]:
# dont forget we need to call the callback method "getInfo" to retrieve the data
df = pd.DataFrame(data, columns=column_df)
# we obtain the data frame as per our demand
# df

In [ ]:
df['date']= pd.to_datetime(df['date_min'], format='%Y%m%d')
df
df_new = df.sort_values(by=['date'])
df_new


,NDVI_mean,NDVI_min,NDVI_max,date_min,id,name,date
0,5989.618644,4776,7516,20230101,1,nepal-crop-poly,2023-01-01
3,5087.623207,4456,5986,20230109,1,nepal-crop-poly,2023-01-09
1,6393.663625,5851,7210,20230117,1,nepal-crop-poly,2023-01-17
4,7439.797914,7134,7748,20230125,1,nepal-crop-poly,2023-01-25
2,7758.353325,7499,7895,20230202,1,nepal-crop-poly,2023-02-02
5,7278.266623,6862,7752,20230210,1,nepal-crop-poly,2023-02-10
6,6741.787484,6524,7110,20230226,1,nepal-crop-poly,2023-02-26
7,6186.423729,6118,6358,20230314,1,nepal-crop-poly,2023-03-14
8,3075.433507,2865,3519,20230330,1,nepal-crop-poly,2023-03-30
9,2388.894394,2259,2826,20230415,1,nepal-crop-poly,2023-04-15


In [ ]:
from scipy.signal import savgol_filter

In [ ]:
import plotly.graph_objects as go

In [ ]:
date = df_new['date'].tolist()
min = savgol_filter(df_new['NDVI_min'].tolist(), 9,2)
mean = savgol_filter(df_new['NDVI_mean'].tolist(), 9,2)
max = savgol_filter(df_new['NDVI_max'].tolist(),9,2)

In [ ]:
# reference taken from following link
# https://stackoverflow.com/a/61494764

fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=date, y=min, name='Minimum NDVI',
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x=date, y=mean, name = 'Mean NDVI',
                         line=dict(color='royalblue', width=4)))
fig.add_trace(go.Scatter(x=date, y=max, name='Max NDVI',
                         line=dict(color='firebrick', width=4) # dash options include 'dash', 'dot', and 'dashdot'
))
# Edit the layout
fig.update_layout(title='Average High and Low Temperatures in New York',
                   xaxis_title='Month',
                   yaxis_title='Temperature (degrees F)')


fig.show()